# Predicting financial market recession based on macroeconomic indicators

## 1. Data Collection and cleaning

For the macroeconomic indicator data, the most used website is world bank followed by FRED(Federal Reserve Bank Of St.Louis).
The website has categories to choose indicators from as decribed above. The free API needs symbol of each indicator to collect data, the list of which is compiled here manually.

Stock market data is collected from yahoo finance and quandl.com as FRED api function in this notebook collects default option of daily stock market data.

Time frame is selected from 1996 to Aug 2020 according to available data.

### Macroeconomic indicators used:
#### 1. Output and Income:

 a. RPI Real Personal Income
 
 b. INDPRO IP Index
 
#### 2. Labor Market
 
 a. CE16OV Civilian Employment
 
 b. UNRATE Civilian Unemployment Rate
 
 c. PAYEMS All Employees: Total nonfarm
 
 d. USGOOD All Employees: Goods-Producing Industries
 
 e. USTPU All Employees: Trade, Transportation & Utilities
 
 
####  3. Real Estate

 a. HOUST Housing Starts: Total New Privately Owned
 
 b. PERMIT New Private Housing Permits (SAAR)
 
#### 4. Consumption and Expenditures

 a. DPCERA3M086SBEA Real personal consumption expenditures
 
 b. CMRMTSPL Real Manu. and Trade Industries Sales
 
 c. RETAIL Retail and Food Services Sales
 
 d. AMTMNO Manufacturers' New Orders: Total Manufacturing
 
 e. AMTMTI Manufacturers' Total Inventories: Total Manufacturing 
 
 f. ACOGNO New Orders for Consumer Goods
 
 g. AMDMUO Unfilled Orders for Durable Goods
 
 h. BUSINV Total Business Inventories 
 
 i. ISRATIO Total Business: Inventories to Sales Ratio
 
#### 5. Money and Credit

 a. M1SL M1 Money Stock
 
 b. M2SL M2 Money Stock
 
 c. TOTRESNS Total Reserves of Depository Institutions
 
 d. BUSLOANS Commercial and Industrial Loans
 
 e. REALLN Real Estate Loans at All Commercial Banks
 
 f. DTCTHFNM Total Consumer Loans and Leases Outstanding
 
#### 6. Interest and Exchange Rates

 a. FEDFUNDS Effective Federal Funds Rate
 
 b. TB3MS 3-Month Treasury Bill
 
 c. TB6MS 6-Month Treasury Bill
 
 d. GS1 1-Year Treasury Rate
 
 e. GS5 5-Year Treasury Rate
 
 f. GS10 10-Year Treasury Rate
 
 g. AAA Moody’s Seasoned Aaa Corporate Bond Yield
 
 h. TB3SMFFM 3-Month Treasury C Minus FEDFUNDS
 
 i. T1YFFM 1-Year Treasury C Minus FEDFUNDS 
 
 j. AAAFFM Moody’s Aaa Corporate Bond Minus FEDFUNDS
 
 k. TWEXAFEGSMTHx Trade Weighted U.S. Dollar Index 
 
 l. EXSZUSx Switzerland / U.S. Foreign Exchange Rate 
 
 l. EXCAUSx Canada / U.S. Foreign Exchange Rate
 
 m. EXUSUKx U.S. / U.K. Foreign Exchange Rate
 
#### 7. Prices

 a. WPSFD49207 PPI: Finished Goods
 
 b. WPSID61 PPI: Intermediate Materials
 
 c. OILPRICEx Crude Oil, spliced WTI and Cushing
 
 d. NAPMPRI ISM Manufacturing: Prices Index
 
 e. CPIAUCSL CPI : All Items
 
#### 8. Stock Market

 a. S&P 500 S&P’s Common Stock Price Index: Composite
 
 b. S&P Dividend yield 
 
 c. NASDAQ Composite Index 
 
 d. S&P P/E ratio
 
 e. Gold ETF

In [ ]:
!pip install fredapi

In [ ]:
import pandas as pd
from fredapi import Fred
fred = Fred(api_key='7ad697af1dec5b19a68ac9ed098f0350')


In [ ]:
ids=['RPI',
 'INDPRO',
 'CE16OV', 
 'UNRATE', 
 'PAYEMS', 
 'USGOOD', 
 'USTPU',
 'HOUST', 
 'PERMIT', 
 'DPCERA3M086SBEA', 
 'CMRMTSPL', 
 'RETAIL', 
 'AMTMTI',
 'AMTMNO',  
 'ACOGNO',
 'AMDMUO',
 'BUSINV',
 'ISRATIO',
 'M1SL',
 'M2SL',
 'TOTRESNS',
 'BUSLOANS',
 'REALLN',
 'DTCTHFNM',
 'FEDFUNDS',
 'TB3MS',
 'TB6MS',
 'GS1',
 'GS5', 
 'GS10',
 'AAA',
 'TB3SMFFM',
 'T1YFFM',
 'AAAFFM',
 'TWEXAFEGSMTH',
 'EXSZUS',
 'EXCAUS',
 'EXUSUK',
 'WPSFD49207',
 'WPSID61',
 'OILPRICE',
 'CPIAUCSL']

In [ ]:
df=pd.DataFrame()
for series_name in ids:
   # print(series_name)
    series_data = fred.get_series(series_name)
    df[series_name]=series_data

In [ ]:
df

Drop columns which have none/missing values in selected time frame, i.e. 1996 to present.

In [ ]:
for col in df.columns.to_list():
    if df['1996-01-01':][col].isnull().any():
        df.drop(col,axis=1,inplace=True)

In [ ]:
df=df['1996-01-01':]

#### Import data for stock market indexes manually using yahoo finance website.

In [ ]:
import pandas as pd
snprets=pd.read_csv(f'../input/us-macroeconomic-data-19962020-source-fred/GSPC (1).csv',header=0,index_col=0,parse_dates=True)


In [ ]:
snprets.drop(['Open','High','Low','Close','Volume'],1,inplace=True)
snprets.rename(columns={'Adj Close':'SP500'},inplace=True)
snprets

In [ ]:
nasrets=pd.read_csv(f"../input/us-macroeconomic-data-19962020-source-fred/IXIC.csv",header=0,index_col=0,parse_dates=True)
nasrets.drop(['Open','High','Low','Close','Volume'],1,inplace=True)
nasrets.rename(columns={'Adj Close':'NASDAQ'},inplace=True)

goldbar=pd.read_csv(f"../input/us-macroeconomic-data-19962020-source-fred/GOLD.csv",index_col=0,parse_dates=True)
goldbar.drop(['Open','High','Low','Close','Volume'],1,inplace=True)
goldbar.rename(columns={'Adj Close':'GOLDBAR'},inplace=True)

In [ ]:
df=pd.concat([df,snprets,nasrets,goldbar],axis=1)

In [ ]:
snpdiv=pd.read_csv(f"../input/us-macroeconomic-data-19962020-source-fred/snpdiv.csv",header=0,index_col=0,parse_dates=True)
snpdiv=snpdiv['1996-01-01':]

PEratio=pd.read_csv(f"../input/us-macroeconomic-data-19962020-source-fred/sp-500-pe-ratio-price-to-earnings-chart.csv",index_col=0,parse_dates=True)
PEratio=PEratio['1996-01-01':]

regime=pd.read_csv(f"../input/us-macroeconomic-data-19962020-source-fred/Recession_Periods.csv",index_col=0,parse_dates=True)
regime=regime['01-01-1996':]

Consistent with the previous works in the literature, we use business cycle dating chronology provided by NBER which involves dates when recession began and ended in US economy. According to NBER's statistics we have 8 recession periods in our dataset where duration is changing from 6 to 18 months. We represent regimes as "Normal" and "Recession" in our dataset.
NBER mentions in chronology dating Aug 2020 update that the peak period in February 2020 is 128 months from previous trough. This it is assumed that March 2020 is a trough, i.e. Recession label and so are following three months.

In [ ]:
df=df['1996-01-01':'2020-06-01']

In [ ]:
df=pd.concat([df,regime],axis=1)

The dates of P/E ratio dataset and dividend yield dataset differ from df.

In [ ]:
df.index = pd.to_datetime(df.index, format="%Y%m").to_period('M')

In [ ]:
snpdiv.index = pd.to_datetime(snpdiv.index, format="%Y%m").to_period('M')

In [ ]:
PEratio['index']=df.index
PEratio.index=PEratio['index']
PEratio=PEratio[' value']
PEratio=pd.DataFrame(PEratio)

In [ ]:
df=pd.concat([df,PEratio,snpdiv],1)

In [ ]:
df=df[:'2020-06']

In [ ]:
df.rename(columns={' value':'P/E'},inplace=True)

In [ ]:
df=df.drop_duplicates(subset='SP500')

In [ ]:
df

Saving the dataframe to a csv file

In [ ]:
df.to_csv(rf"./output_macrodata.csv")

In [ ]:
macdf=pd.read_csv(r"./output_macrodata.csv",index_col=0,parse_dates=True)